In [ ]:
import requests

def get_mails(access_token):
    # 定义请求参数
    endpoint = "https://graph.microsoft.com/v1.0/me/messages"

    # 构造请求头
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }

    try:
        # 发送GET请求
        response = requests.get(endpoint, headers=headers)
        response.raise_for_status()  # 检查HTTP错误

        # 解析响应数据
        messages = response.json().get("value", [])
        # print(messages)
        # 打印邮件主题
        return messages

    except requests.exceptions.HTTPError as err:
        print(f"HTTP错误: {err}")
        if response.status_code == 401:
            print("Token可能已过期或权限不足")
        elif response.status_code == 403:
            print("缺少邮件读取权限 (Mail.Read)")
    except Exception as e:
        print(f"其他错误: {e}")




In [ ]:
access_token = "EwA4BMl6BAAUBKgm8k1UswUNwklmy2v7U/S+1fEAAXZ4GvFX/eRC1NDgZX/ipBNkKZJjhOK0gW5SQpLckw5SBAapEv179Sj4JxVY6EDNhvfOZij6eAK6AmEYnGqzBJdckpTGKV0zFc+5Qxc6/z4ZwV93Gv1RCMECqIPMXMMA7Obd/3UZhguyGjGZfj5PnjFWF883hnvAdTvGoomX/2Z6e/vYYY1ZT1Tw44RlmCvEdtp82kNNcTBaooJ1w26gbNh+Xk4qjRexAZVrBQZtHfv/qEHvY/EEFVT6Twv7FaJwYPv35NKwoOtQzD6dk/qPoV2H/XitG6JcEgOHJcBhRxe7uppFRBLPZC9Mm5m61nZRT/LtuE094eVEtw+7V//yOJAQZgAAEIu8pEALRQjxFGk6HZv2ScIAA3XW5L8y0bYQ8xw6FXIMr2NM+xyuv1vj/XPS26R91BaodCNaHL6iaxxqTjsxPwRMq++6a5dWyiw+3wj8WdXoTxagP/JefXVd3uRWtCrsTsCRLjJDbOBJR63Kmna47JA2ij+LKeLt9w7mWVuTaSRbSKWo2rFNbQRQFQHYs6knAu1qbso4mL4xg4NMbcJYpKTQ2wdIDQrDgLFO+bBWnot0YFfJt/TLGZFDnyaqg4gJILI86E6W+ir1G6GsvXbTgw+isRcaqNujipbth7+4OrxjHGFYiKLZd81BGYQr2F31QZUVIWDUM9G3Gpv9QuMRG/s+lpbIxWqjG6wdA/GThMxQX2NHW8EdINfubAUJUxmYtPGOYF6xTQif1wMVqjjlGN+qpX+ft0XtvErb5YcUx7gLLnoBH7RxQI8mK/Nt17u6IPfBYqO5/jh8ybU4LkYYMiEo9bRXmIel/Z6mGj/iNmAzlkOAdhx2A75ffa92ISqO3u9IqeF2T6z/pxVXTdQUgX0y1JQoVwYdTIVeD7L7BOJg7RiQHAtDiH9fUcUPGFOJWG0ogk22TJIAtvkfvbBPxJvz1e8heG2CA/BXa7/ymu2eJbdp9r71neZq9Fke4F+XTniXErh2bBCH2GDGLYvMOrEUtIXr72M9ONRR3/3MnWbQPelHvi7aSy8RkXj78RyHlbz+KlSTzl5IawD5mxnDw4XC1O5oPHNIm+NDWrLEI8FTYFUK8x1ZdijT25INOtWRb/Fe8FVrjllNefhmsGqoXtv6KoxW/iT2l8vrEbUENVF65y3koelMnP1wtPbiXROIZ7JP0pCCKeNNPs0khRsftrhXz4fUFLYyWgJ95wkPlvam1g8OeIqGF6fOpAAvaa0W0EqN+W6OG3DIJO8uil8v8/eaSOvXYbyRrNMmwEtyFfvM2x6J9V9X+e6gGLS5Z8aXE4SoHIHn06EjnE7MAQyk40TlxkW9I1y3UYUDkuLaZcNKCnhRw2XcSRcpQcHJ4XrJt5G5nm8JoIiFXLyEpDZwJ6+uSkID"

messages = get_mails(access_token)
for msg in messages:
            print(f"邮件ID: {msg.get('id')}")
            print(f"主题: {msg.get('subject', '无主题')}")
            print(f"发件人: {msg.get('from', {}).get('emailAddress', {}).get('address')}")
            print("-" * 50)